In [13]:
import cv2
import numpy as np
import os

In [2]:
# Load the image
path = os.path.join(r"C:\Users\reema\Downloads\img.jpg")
img = cv2.imread(path)

# SCALING

In [24]:
# Define the scale factor
scale_factor = 0.5

In [25]:
# Scale the image
scaled_img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

In [27]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

In [28]:
# Detect keypoints and compute descriptors
kp1, des1 = sift.detectAndCompute(img, None)
kp2, des2 = sift.detectAndCompute(scaled_img, None)

In [29]:
# Initialize BFMatcher with default params
bf = cv2.BFMatcher()

In [30]:
# Match descriptors
matches = bf.knnMatch(des1, des2, k=2)


In [31]:
# Apply ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.50 * n.distance:
        good_matches.append(m)


In [ ]:
# Draw matches
matching_img = cv2.drawMatches(img, kp1, scaled_img, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [ ]:
# Show the result
cv2.imshow('Matching Result', matching_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Rotation

In [ ]:
# Define rotation angle in degrees
angle = 30


In [ ]:
# Define rotation matrix
rotation_matrix = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), angle, 1)


In [ ]:
# Apply rotation to image
rotated_img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))

In [ ]:
# Extract SIFT keypoints and descriptors from original and rotated images
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(img, None)
keypoints2, descriptors2 = sift.detectAndCompute(rotated_img, None)


In [ ]:
# Display original and rotated images with keypoints
img_with_keypoints1 = cv2.drawKeypoints(img, keypoints1, None)
img_with_keypoints2 = cv2.drawKeypoints(rotated_img, keypoints2, None)
cv2.imshow("Original Image with Keypoints", img_with_keypoints1)
cv2.imshow("Rotated Image with Keypoints", img_with_keypoints2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Affine

In [ ]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

# Find keypoints and descriptors
kp, des = sift.detectAndCompute(img, None)


In [ ]:
# Define the affine transformation matrix
theta = np.radians(45)
scale = 1.0
tx, ty = 100, -50

M = np.array([
    [scale * np.cos(theta), -scale * np.sin(theta), tx],
    [scale * np.sin(theta), scale * np.cos(theta), ty]
])

In [ ]:
# Apply the affine transformation
img_affine = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

In [ ]:
# Find keypoints and descriptors in the transformed image
kp_affine, des_affine = sift.detectAndCompute(img_affine, None)

In [ ]:
# Draw keypoints on the images
img_kp = cv2.drawKeypoints(img, kp, None)
img_affine_kp = cv2.drawKeypoints(img_affine, kp_affine, None)

In [ ]:
# Show the images
cv2.imshow('Original', img_kp)
cv2.imshow('Affine Transformation', img_affine_kp)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Perspective

In [ ]:
# Define the four corners of the original image
pts1 = np.float32([[0, 0], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1], [img.shape[1]-1, 0]])


In [4]:
# Define the four corners of the desired output image
pts2 = np.float32([[0, 0], [0, 400], [400, 400], [400, 0]])

In [5]:
# Calculate the perspective transform matrix
M = cv2.getPerspectiveTransform(pts1, pts2)

In [6]:
# Apply the perspective transform to the original image
perspective_img = cv2.warpPerspective(img, M, (400, 400))


In [7]:
# Extract SIFT keypoints and descriptors from original and transformed images
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(img, None)
keypoints2, descriptors2 = sift.detectAndCompute(perspective_img, None)

In [8]:
# Use a brute-force matcher to find matches between descriptors
bf = cv2.BFMatcher()
matches = bf.match(descriptors1, descriptors2)

In [20]:
# Filter matches by distance threshold
threshold = 0.6 * np.max([m.distance for m in matches])
good_matches = [m for m in matches if m.distance < threshold]

In [21]:
# Draw matching lines on a new image
match_img = cv2.drawMatches(img, keypoints1, perspective_img, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [22]:
# Display image with matching lines
cv2.imshow("Matches", match_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
